In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time
from IPython.display import display, clear_output
import threading

In [8]:
# Quick test function to check camera availability
def test_camera():
    """Test if camera is available and working"""
    print("🔍 Testing camera availability...")
    
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("❌ Camera not available")
        print("Possible solutions:")
        print("- Check if camera is connected")
        print("- Close other applications using the camera")
        print("- Try different camera index (1, 2, etc.)")
        return False
    
    ret, frame = cap.read()
    cap.release()
    
    if ret:
        print("✅ Camera is working!")
        print(f"Frame size: {frame.shape[1]}x{frame.shape[0]}")
        return True
    else:
        print("❌ Camera detected but can't capture frames")
        return False

# Test camera first
camera_ok = test_camera()

🔍 Testing camera availability...
✅ Camera is working!
Frame size: 1280x720


In [4]:
# Method 1: OpenCV Haar Cascade Approach (Simple & Fast)
class OpenCVSmileDetector:
    def __init__(self):
        """Initialize OpenCV-based smile detector"""
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        self.smile_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_smile.xml')
        
        # Statistics
        self.total_frames = 0
        self.smile_frames = 0
        self.start_time = time.time()
        
    def detect_smile(self, frame):
        """Detect smile in a frame"""
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = self.face_cascade.detectMultiScale(gray, 1.3, 5)
        
        smile_detected = False
        
        for (x, y, w, h) in faces:
            # Draw face rectangle
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            
            # Region of interest for smile detection
            roi_gray = gray[y:y+h, x:x+w]
            
            # Detect smiles within face region
            smiles = self.smile_cascade.detectMultiScale(roi_gray, 1.8, 20)
            
            if len(smiles) > 0:
                smile_detected = True
                # Draw smile rectangles
                for (sx, sy, sw, sh) in smiles:
                    cv2.rectangle(frame[y:y+h, x:x+w], (sx, sy), (sx+sw, sy+sh), (0, 255, 0), 2)
                
                # Add smile text
                cv2.putText(frame, '😊 SMILING!', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            else:
                cv2.putText(frame, '😐 NOT SMILING', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
        
        # Update statistics
        self.total_frames += 1
        if smile_detected:
            self.smile_frames += 1
            
        return frame, smile_detected, len(faces)
    
    def get_statistics(self):
        """Get current detection statistics"""
        elapsed_time = time.time() - self.start_time
        smile_percentage = (self.smile_frames / max(self.total_frames, 1)) * 100
        fps = self.total_frames / max(elapsed_time, 1)
        
        return {
            'elapsed_time': elapsed_time,
            'total_frames': self.total_frames,
            'smile_frames': self.smile_frames,
            'smile_percentage': smile_percentage,
            'fps': fps
        }

print("✅ OpenCV Smile Detector ready!")

✅ OpenCV Smile Detector ready!


In [5]:
# Real-time Webcam Smile Detection Function
def start_realtime_smile_detection(method='opencv', show_stats=True, save_screenshots=False):
    """
    Start real-time smile detection using webcam
    
    Parameters:
    - method: 'opencv' or 'mediapipe'
    - show_stats: Display real-time statistics
    - save_screenshots: Save screenshots when smiling
    """
    
    # Initialize detector based on method
    if method == 'opencv':
        detector = OpenCVSmileDetector()
        print("🎥 Starting OpenCV-based smile detection...")
    # else:
    #     detector = MediaPipeSmileDetector()
    #     print("🎥 Starting MediaPipe-based smile detection...")
    
    # Initialize camera
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("❌ Error: Could not open camera")
        print("Make sure your webcam is connected and not being used by another application")
        return
    
    # Set camera properties for better performance
    # cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    # cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    # cap.set(cv2.CAP_PROP_FPS, 30)
    
    print("📋 Controls:")
    print("  - Press 'q' to quit")
    print("  - Press 's' to save screenshot")
    print("  - Press 'r' to reset statistics")
    print("  - Press SPACE to toggle statistics display")
    
    screenshot_count = 0
    show_stats_flag = show_stats
    
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("❌ Failed to grab frame from camera")
                break
            
            # Flip frame horizontally for mirror effect
            frame = cv2.flip(frame, 1)
            
            # Detect smile based on selected method
            if method == 'opencv':
                processed_frame, smile_detected, faces_count = detector.detect_smile(frame)
            else:
                processed_frame, smile_detected, confidence, faces_count = detector.detect_smile(frame)
            
            # Get current statistics
            stats = detector.get_statistics()
            
            # Add statistics overlay if enabled
            if show_stats_flag:
                y_offset = processed_frame.shape[0] - 120
                cv2.rectangle(processed_frame, (10, y_offset-10), (400, processed_frame.shape[0]-10), (0, 0, 0), -1)
                
                cv2.putText(processed_frame, f"Frames: {stats['total_frames']}", (20, y_offset + 20), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
                cv2.putText(processed_frame, f"Smiles: {stats['smile_frames']}", (20, y_offset + 40), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
                cv2.putText(processed_frame, f"Smile %: {stats['smile_percentage']:.1f}%", (20, y_offset + 60), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
                cv2.putText(processed_frame, f"FPS: {stats['fps']:.1f}", (20, y_offset + 80), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
                cv2.putText(processed_frame, f"Faces: {faces_count}", (20, y_offset + 100), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
            
            # Display the frame
            cv2.imshow('Real-time Smile Detection', processed_frame)
            
            # Auto-save screenshot when smiling (if enabled)
            if save_screenshots and smile_detected and stats['total_frames'] % 30 == 0:  # Every 30 frames
                screenshot_path = f'smile_screenshot_{screenshot_count}.jpg'
                cv2.imwrite(screenshot_path, processed_frame)
                print(f"📸 Auto-saved: {screenshot_path}")
                screenshot_count += 1
            
            # Handle key presses
            key = cv2.waitKey(1) & 0xFF
            
            if key == ord('q'):
                break
            elif key == ord('s'):
                # Manual screenshot
                screenshot_path = f'manual_screenshot_{screenshot_count}.jpg'
                cv2.imwrite(screenshot_path, processed_frame)
                print(f"📸 Screenshot saved: {screenshot_path}")
                screenshot_count += 1
            elif key == ord('r'):
                # Reset statistics
                if method == 'opencv':
                    detector = OpenCVSmileDetector()
                else:
                    detector = MediaPipeSmileDetector()
                print("🔄 Statistics reset!")
            elif key == ord(' '):
                # Toggle statistics display
                show_stats_flag = not show_stats_flag
                print(f"📊 Statistics display: {'ON' if show_stats_flag else 'OFF'}")
    
    except KeyboardInterrupt:
        print("\n⏹️ Detection stopped by user")
    
    finally:
        # Cleanup
        cap.release()
        cv2.destroyAllWindows()
        
        # Print final statistics
        final_stats = detector.get_statistics()
        print("\n📊 FINAL STATISTICS:")
        print("=" * 40)
        print(f"Total time: {final_stats['elapsed_time']:.1f} seconds")
        print(f"Total frames: {final_stats['total_frames']}")
        print(f"Frames with smile: {final_stats['smile_frames']}")
        print(f"Smile percentage: {final_stats['smile_percentage']:.1f}%")
        print(f"Average FPS: {final_stats['fps']:.1f}")
        if method == 'mediapipe':
            print(f"Average confidence: {final_stats['avg_confidence']:.3f}")
        print(f"Screenshots saved: {screenshot_count}")

print("🚀 Real-time smile detection function ready!")

🚀 Real-time smile detection function ready!


In [6]:
# Start real-time smile detection
if camera_ok:
    print("Choose detection method:")
    print("1. OpenCV (faster, simpler)")
    print("2. MediaPipe (more accurate, advanced)")
    
    # For automatic start, uncomment one of these:
    
    # Option 1: OpenCV method
    start_realtime_smile_detection(method='opencv', show_stats=True, save_screenshots=True)
    
    # Option 2: MediaPipe method  
    # start_realtime_smile_detection(method='mediapipe', show_stats=True, save_screenshots=False)
    
    print("💡 Uncomment one of the lines above to start detection!")
    print("   - Use 'opencv' for faster detection")
    print("   - Use 'mediapipe' for more accurate detection")
else:
    print("❌ Cannot start smile detection - camera not available")

Choose detection method:
1. OpenCV (faster, simpler)
2. MediaPipe (more accurate, advanced)
🎥 Starting OpenCV-based smile detection...
📋 Controls:
  - Press 'q' to quit
  - Press 's' to save screenshot
  - Press 'r' to reset statistics
  - Press SPACE to toggle statistics display
📸 Screenshot saved: manual_screenshot_0.jpg
📸 Screenshot saved: manual_screenshot_1.jpg
📸 Screenshot saved: manual_screenshot_2.jpg
🔄 Statistics reset!
🔄 Statistics reset!
🔄 Statistics reset!
🔄 Statistics reset!
📸 Screenshot saved: manual_screenshot_3.jpg

📊 FINAL STATISTICS:
Total time: 28.2 seconds
Total frames: 407
Frames with smile: 0
Smile percentage: 0.0%
Average FPS: 14.5
Screenshots saved: 4
💡 Uncomment one of the lines above to start detection!
   - Use 'opencv' for faster detection
   - Use 'mediapipe' for more accurate detection
